In [2]:
import pandas as pd
import numpy as np

In [5]:
data=pd. read_csv("./data/train.csv")

In [ ]:
data.drop(labels=["id"],axis=1,inplace=True)



In [7]:
X=data.drop(labels=["price"],axis=1)
    

In [6]:
y=data[["price"]]

In [8]:
categorical_cols=X.select_dtypes(include='object').columns


In [9]:
numerical_cols=X.select_dtypes(exclude='object').columns


In [10]:
numerical_cols


Index(['id', 'carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [11]:
X.select_dtypes(include='object')


,cut,color,clarity
0,Premium,F,VS2
1,Very Good,J,SI2
2,Ideal,G,VS1
3,Ideal,G,VS1
4,Premium,G,VS2
...,...,...,...
193568,Ideal,D,VVS2
193569,Premium,G,VVS2
193570,Very Good,F,SI1
193571,Very Good,D,SI1


In [12]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']


In [13]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [15]:
num_pipeline=Pipeline(
    
    
    steps=[
        
        ('imputer',SimpleImputer()),
        ('scaler', StandardScaler())
    ]
    
    
)

In [14]:
cat_pipeline=Pipeline(
    
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
    
)

In [16]:
preprocessor=ColumnTransformer(
    [
        
        ('num_pipeline',num_pipeline,numerical_cols),
        ('cat_pipeline',cat_pipeline,categorical_cols)
    ]
)
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [17]:
preprocessor.fit_transform(X_train)


array([[ 1.27491908, -0.97543926, -0.84960654, ...,  4.        ,
         5.        ,  5.        ],
       [-1.10290693,  0.2351953 ,  1.83363716, ...,  1.        ,
         1.        ,  2.        ],
       [-0.99761317,  0.49461699,  0.81585507, ...,  3.        ,
         3.        ,  4.        ],
       ...,
       [ 0.03428   ,  0.45138004,  1.55606023, ...,  1.        ,
         3.        ,  2.        ],
       [-0.87327749,  0.66756478, -1.77486298, ...,  4.        ,
         3.        ,  4.        ],
       [ 1.3030439 ,  0.25681377,  0.81585507, ...,  4.        ,
         3.        ,  2.        ]])

In [18]:
preprocessor.transform(X_test)


array([[-0.47300385, -0.56468825, -0.94213218, ...,  3.        ,
         1.        ,  3.        ],
       [-0.57300521, -0.17555571,  1.00090636, ...,  2.        ,
         4.        ,  2.        ],
       [ 1.58109167, -1.06191316,  0.2607012 , ...,  4.        ,
         4.        ,  7.        ],
       ...,
       [-0.93195878, -0.19717419, -3.34779894, ...,  3.        ,
         6.        ,  3.        ],
       [-1.198492  , -0.82410994, -0.20192702, ...,  4.        ,
         3.        ,  2.        ],
       [-0.61614902,  2.61322747, -0.75708089, ...,  3.        ,
         6.        ,  3.        ]])

In [19]:
preprocessor.get_feature_names_out()


array(['num_pipeline__id', 'num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [20]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())


In [21]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [22]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [23]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [24]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [25]:
list(models)


['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [26]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    print(model)


LinearRegression()
Lasso()
Ridge()
ElasticNet()


In [27]:
models.keys()


dict_keys(['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet'])

In [28]:
models.values()


dict_values([LinearRegression(), Lasso(), Ridge(), ElasticNet()])

In [29]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1013.904802565844
MAE: 674.0257224619529
R2 score 93.68908132630604


Lasso
Model Training Performance
RMSE: 1013.8723151331257
MAE: 675.2036875769029
R2 score 93.68948574743231


Ridge
Model Training Performance
RMSE: 1013.9059934108718
MAE: 674.0567268148293
R2 score 93.6890665017771


Elasticnet
Model Training Performance
RMSE: 1513.91821834484
MAE: 1050.754957945767
R2 score 85.92970983426311




In [30]:
trained_model_list


[]

In [31]:
model_list


['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']